In [ ]:

import cv2
import os
import pandas as pd
from datetime import datetime
import time

# Import DeepFace with error handling
try:
    from deepface import DeepFace
except ImportError:
    print("ERROR: DeepFace not installed!")
    print("Please run: pip install deepface")
    exit()

# Configuration
DATABASE_FOLDER = "employee_database"
ATTENDANCE_FILE =  r"C:\Users\USER\Desktop\Flask\stressaje\office_attendance.csv"

TEMP_IMAGE = "temp_capture.jpg"
RECOGNITION_THRESHOLD = 0.7
FRAME_SKIP = 60  
COOLDOWN = 5  # Seconds between recognition attempts

# Create database folder if needed
if not os.path.exists(DATABASE_FOLDER):
    os.makedirs(DATABASE_FOLDER)
    print(f"Created '{DATABASE_FOLDER}' folder")
    print("Add employee photos (format: Name.jpg)")
    print("Then run the program again.")
    exit()

# Check if database has photos
db_files = [f for f in os.listdir(DATABASE_FOLDER) 
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

if len(db_files) == 0:
    print(f"ERROR: No photos in '{DATABASE_FOLDER}' folder!")
    print("Add employee photos (format: Name.jpg)")
    exit()

# Create attendance CSV file if it doesn't exist
if not os.path.exists(ATTENDANCE_FILE):
    df = pd.DataFrame(columns=['Employee', 'Date', 'Time', 'Status'])
    df.to_csv(ATTENDANCE_FILE, index=False)

# Load employees
print("=" * 60)
print("       DEEPFACE OFFICE ATTENDANCE SYSTEM")
print("=" * 60)
print("\nLoading employees...")

employees = []
for file in db_files:
    name = os.path.splitext(file)[0]
    employees.append(name)
    print(f"  ✓ {name}")

print(f"\nTotal: {len(employees)} employees")

# Initialize tracking
checked_in = set()
last_check = 0

# Start camera
print("\n" + "=" * 60)
print("SYSTEM READY - Press 'q' to quit")
print("=" * 60 + "\n")

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("ERROR: Cannot open camera!")
    exit()

# Set camera resolution
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

frame_num = 0
processing = False

# Main loop
try:
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("ERROR: Cannot read frame")
            break
        
        frame_num += 1
        now = time.time()
        
        # FOR THE Display status
        color = (0, 165, 255) if processing else (0, 255, 0)
        cv2.putText(frame, "DeepFace Attendance", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        cv2.putText(frame, f"Checked: {len(checked_in)}/{len(employees)}", 
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        status = "Processing..." if processing else "Ready"
        cv2.putText(frame, status, (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Check if should process
        can_process = (frame_num % FRAME_SKIP == 0 and 
                       not processing and 
                       (now - last_check) > COOLDOWN)
        
        if can_process:
            processing = True
            cv2.imwrite(TEMP_IMAGE, frame)
            
            try:
                print(" Searching for face...")
                
                # DeepFace search
                result = DeepFace.find(
                    img_path=TEMP_IMAGE,
                    db_path=DATABASE_FOLDER,
                    model_name="VGG-Face",
                    enforce_detection=False,
                    silent=True
                )
                
                # Process results
                matched = False
                
                if result is not None and len(result) > 0:
                    df_result = result[0] if isinstance(result, list) else result
                    
                    if isinstance(df_result, pd.DataFrame) and not df_result.empty:
                        top_match = df_result.iloc[0]
                        
                        # Get identity (file path)
                        if 'identity' in top_match:
                            path = top_match['identity']
                        else:
                            print("    No identity column found")
                            continue
                        
                        # Get distance - flexible column detection
                        dist = None
                        possible_cols = ['VGG-Face_cosine', 'distance', 'VGG-Face_euclidean']
                        
                        for col in possible_cols:
                            if col in top_match:
                                dist = top_match[col]
                                break
                        
                        if dist is None:
                            # Use last numeric column as fallback
                            numeric_cols = df_result.select_dtypes(include=['float64', 'int64']).columns
                            if len(numeric_cols) > 0:
                                dist = top_match[numeric_cols[-1]]
                            else:
                                print("    Cannot find distance column")
                                continue
                        
                        # Extract employee name
                        name = os.path.splitext(os.path.basename(path))[0]
                        
                        # Calculate confidence
                        conf = max(0, min(100, (1 - float(dist)) * 100))
                        
                        print(f"  Found: {name} (Confidence: {conf:.1f}%)")
                        
                        # Check if match is good enough
                        if float(dist) < RECOGNITION_THRESHOLD:
                            if name not in checked_in:
                                # Mark attendance
                                checked_in.add(name)
                                
                                dt = datetime.now()
                                date_str = dt.strftime('%Y-%m-%d')
                                time_str = dt.strftime('%H:%M:%S')
                                
                                # Save to CSV
                                entry = pd.DataFrame({
                                    'Employee': [name],
                                    'Date': [date_str],
                                    'Time': [time_str],
                                    'Status': ['Present']
                                })
                                
                                entry.to_csv(ATTENDANCE_FILE, mode='a', 
                                           header=False, index=False)
                                
                                print(f"   MARKED: {name} at {time_str}\n")
                                matched = True
                            else:
                                print(f"    {name} already checked in\n")
                                matched = True
                        else:
                            print(f"    Match too weak (distance: {dist:.3f})\n")
                    else:
                        print("   Empty result\n")
                else:
                    print("   No face detected\n")
                    
            except Exception as e:
                print(f"   Error: {str(e)}\n")
            
            finally:
                # Cleanup
                if os.path.exists(TEMP_IMAGE):
                    try:
                        os.remove(TEMP_IMAGE)
                    except:
                        pass
                
                processing = False
                last_check = now
        
        # Display frame
        cv2.imshow('Attendance System', frame)
        
        # Check for quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("\n Shutting down...")
            break

except KeyboardInterrupt:
    print("\n  Interrupted by user")

finally:
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    
    if os.path.exists(TEMP_IMAGE):
        try:
            os.remove(TEMP_IMAGE)
        except:
            pass

# Display summary
print("\n" + "=" * 60)
print("                  ATTENDANCE SUMMARY")
print("=" * 60)
print(f"Database: {len(employees)} employees")
print(f"Checked In: {len(checked_in)} employees")

if len(employees) > 0:
    rate = (len(checked_in) / len(employees)) * 100
    print(f"Attendance Rate: {rate:.1f}%")

if checked_in:
    print("\n Present:")
    for emp in sorted(checked_in):
        print(f"   • {emp}")
    
    missing = set(employees) - checked_in
    if missing:
        print("\n Absent:")
        for emp in sorted(missing):
            print(f"   • {emp}")
else:
    print("\nNo attendance recorded")

print(f"\n Saved to: {ATTENDANCE_FILE}")
print("=" * 60)


ERROR: No photos in 'employee_database' folder!
Add employee photos (format: Name.jpg)
       DEEPFACE OFFICE ATTENDANCE SYSTEM

Loading employees...

Total: 0 employees

SYSTEM READY - Press 'q' to quit

 Searching for face...
  Found: Bill_Gates_0001 (Confidence: 33.7%)
   Error: [Errno 13] Permission denied: 'C:\\Users\\USER\\Desktop\\Flask\\stressaje\\office_attendance.csv'

 Searching for face...
   Empty result

 Searching for face...
   Empty result

 Searching for face...
   Empty result

 Searching for face...
   Empty result

 Searching for face...
  Found: Adam_Rich_0001 (Confidence: 74.9%)
   Error: [Errno 13] Permission denied: 'C:\\Users\\USER\\Desktop\\Flask\\stressaje\\office_attendance.csv'

 Searching for face...
  Found: Adam_Rich_0001 (Confidence: 63.0%)
    Adam_Rich_0001 already checked in

 Searching for face...
   Empty result

 Searching for face...
  Found: Bill_Gates_0001 (Confidence: 34.7%)
    Bill_Gates_0001 already checked in


 Shutting down...

       

: 